# Create z-space and y-space interpolation gifs

Additional dependencies:
```
imageio
```

Hard-coded to handle vae at the moment. Will need to reimplement once I add the other models.

In [83]:
import os
import sys
import argparse
from codebase import args as codebase_args
from pprint import pprint
import tensorflow as tf
from glob import glob
import numpy as np
import imageio

# Settings
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--src',    type=str,   default='svhn',    help="Src data")
parser.add_argument('--trg',    type=str,   default='extra',   help="Trg data")
parser.add_argument('--design', type=str,   default='v1',      help="design")
parser.add_argument('--gw',     type=float, default=.01,       help="Gen weight")
parser.add_argument('--rw',     type=float, default=0.5,       help="Rec weight")
parser.add_argument('--npc',    type=int,   default=None,      help="NPC")
parser.add_argument('--lr',     type=float, default=3e-3,      help="Learning rate")
parser.add_argument('--run',    type=int,   default=999,       help="Run index")
parser.add_argument('--logdir', type=str,   default='log',     help="Log directory")
codebase_args.args = args = parser.parse_args([])

In [36]:
def t2u(x):
    """
    Convert 'tanh' encoding to 'uint-8' encoding
    """
    s = (x + 1) / 2
    u = (s * 255).astype('uint8')
    return u

def build_model(design):
    args.design = design

    from codebase.models.vae import vae
    from codebase.train import train
    from codebase.utils import get_data

    M = vae()
    M.sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    return M, saver

In [37]:
M, saver = build_model('v2')

In [38]:
path = glob('checkpoints/*extra*v2_gw*')[0]
path = tf.train.latest_checkpoint(path)
saver.restore(M.sess, path)

INFO:tensorflow:Restoring parameters from checkpoints/model=vae_src=svhn_trg=extra_design=v2_gw=1e-02_rw=5e-01_npc=100_lr=3e-03_run=0000/model-160000


INFO:tensorflow:Restoring parameters from checkpoints/model=vae_src=svhn_trg=extra_design=v2_gw=1e-02_rw=5e-01_npc=100_lr=3e-03_run=0000/model-160000


In [104]:
def generate_img_by_z(z, n=10):
    z = z.reshape(n, 100)
    z = np.tile(z, (100 / n, 1))
    y = np.tile(np.eye(10), (10, 1)).T.reshape(-1, 10)
    
    x = t2u(M.fn_fake_x(z, y))
    img = x.reshape(10, 10, 32, 32, 3).swapaxes(1, 2).reshape(320, 320, 3)
    return img    

def generate_img_by_y(y, z_fix):
    y = y.reshape(1, 10)
    y = np.tile(y, (100, 1))
    x = t2u(M.fn_fake_x(z_fix, y))
    img = x.reshape(10, 10, 32, 32, 3).swapaxes(1, 2).reshape(320, 320, 3)
    return img    

def generate_seq(z0, z1):
    ts = (np.cos(np.linspace(0, np.pi, 10)) + 1) / 2
    zs = [t * z0 + (1 - t) * z1 for t in ts]
    return zs

In [108]:
n = 10
z_keyframes = [np.random.randn(n, 100) for i in xrange(10)]
z_keyframes += [z_keyframes[0]]

zs = []
for i in xrange(10):
    z_interps = generate_seq(z_keyframes[i], z_keyframes[i + 1])
    for z in z_interps:
        zs += [z]
        
imgs = []
for z in zs:
    imgs += [generate_img_by_z(z, n=n)]

In [110]:
imageio.mimwrite('interp_by_z.gif', imgs , fps = 10)

In [111]:
z_fix = np.random.randn(100, 100)
y_keyframes = [np.eye(10)[i] for i in xrange(10)]
y_keyframes += [y_keyframes[0]]

ys = []
for i in xrange(10):
    y_interps = generate_seq(y_keyframes[i], y_keyframes[i + 1])
    for y in y_interps:
        ys += [y]
        
imgs = []
for y in ys:
    imgs += [generate_img_by_y(y, z_fix=z_fix)]

In [112]:
imageio.mimwrite('interp_by_y.gif', imgs , fps = 10)